# 1. 讀取檔案

In [221]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np

# 使用 pandas 的 read_csv 函式讀取訓練資料
df = pd.read_csv("data/train.csv")

In [222]:
df.head(10)


,id,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,name
0,3,16.0,8,304.0,150.0,3433,12.0,70,usa,amc rebel sst
1,18,27.0,4,97.0,88.0,2130,14.5,70,japan,datsun pl510
2,376,37.0,4,91.0,68.0,2025,18.2,82,japan,mazda glc custom l
3,248,36.1,4,91.0,60.0,1800,16.4,78,japan,honda civic cvcc
4,177,23.0,4,115.0,95.0,2694,15.0,75,europe,audi 100ls
5,63,14.0,8,400.0,175.0,4385,12.0,72,usa,pontiac catalina
6,246,32.8,4,78.0,52.0,1985,19.4,78,japan,mazda glc deluxe
7,352,29.9,4,98.0,65.0,2380,20.7,81,usa,ford escort 2h
8,108,20.0,4,97.0,88.0,2279,19.0,73,japan,toyota carina
9,90,12.0,8,429.0,198.0,4952,11.5,73,usa,mercury marquis brougham


# 2.資料預處理

In [223]:
#model_year的分類
# def lam(x):
#     if 70<=x<=74 :return 1
#     elif 75<=x<=78:return 2
#     else:return 3
def classify(df):
    df['model_year']=pd.qcut(df['model_year'],3,labels=False)
    # df['model_year']=df['model_year'].map(lam,3)
    return df
df=(classify(df))

In [224]:
def FindCom(df):
    comp=[i.split(" ")[0] for i in df['name']]
    return comp

def Firm(df):
    comp=FindCom(df)
    # print(set(comp))
    df['name']=comp
    df['name']=df['name'].fillna(df['name'].mode())
    data_dum = pd.get_dummies(df['name'])
    df=pd.concat([df,data_dum],axis=1)
    df['name']=df['name'].fillna(df['name'].mode())
    return df
df_clean=Firm(df)




In [225]:
#嘗試LableEncoding失敗
# def Firm2(df):
#     comp=FindCom(df)
#     # print(set(comp))
#     df['name']=comp
#     from sklearn.preprocessing import LabelEncoder
#     le=LabelEncoder()
#     df['name']=le.fit_transform(df['name'])
#     return df
# df_clean=Firm2(df)

In [226]:

# df_clean

In [227]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 318 entries, 0 to 317
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            318 non-null    int64  
 1   mpg           318 non-null    float64
 2   cylinders     318 non-null    int64  
 3   displacement  318 non-null    float64
 4   horsepower    313 non-null    float64
 5   weight        318 non-null    int64  
 6   acceleration  318 non-null    float64
 7   model_year    318 non-null    int64  
 8   origin        318 non-null    object 
 9   name          318 non-null    object 
dtypes: float64(4), int64(4), object(2)
memory usage: 25.0+ KB
None


In [228]:
m=df.iloc[:,1:8].corr()
print(m)

                   mpg  cylinders  displacement  horsepower    weight  \
mpg           1.000000  -0.770459     -0.801543   -0.772308 -0.827333   
cylinders    -0.770459   1.000000      0.951825    0.839187  0.893808   
displacement -0.801543   0.951825      1.000000    0.888972  0.929479   
horsepower   -0.772308   0.839187      0.888972    1.000000  0.855325   
weight       -0.827333   0.893808      0.929479    0.855325  1.000000   
acceleration  0.390101  -0.479743     -0.512604   -0.672726 -0.380938   
model_year    0.564872  -0.328503     -0.336201   -0.385714 -0.278372   

              acceleration  model_year  
mpg               0.390101    0.564872  
cylinders        -0.479743   -0.328503  
displacement     -0.512604   -0.336201  
horsepower       -0.672726   -0.385714  
weight           -0.380938   -0.278372  
acceleration      1.000000    0.258433  
model_year        0.258433    1.000000  


In [229]:
def RemoveIqrOutliers(df,colume,k=1.5):
    # 計算四分位數 (Q1, Q3)
    Q1 = df[colume].quantile(0.25)
    Q3 = df[colume].quantile(0.75)
    
    # 計算四分位距 (IQR)
    IQR = Q3 - Q1
    
    # 定義異常值邊界
    lower_bound = Q1 - k * IQR
    upper_bound = Q3 + k * IQR
    print(len(df))
    # 5. 根據邊界過濾數據
    # 僅保留那些在 [lower_bound, upper_bound] 範圍內的數據點
    Outer_df = df[(df[colume] < lower_bound) | (df[colume] > upper_bound)]
    df=df.drop(index=Outer_df.index)
    print(df)
    print(len(df))
    return df

In [230]:
def STD(df_clean,i,k=3):
    u=-1
    while u!=len(df_clean[i]):
        u=len(df_clean[i])
        print(len(df_clean))
        desc=df_clean[i].describe()
        std=desc['std']
        mean=desc['mean']
        up=mean+k*std
        lower=mean-k*std
        Outer_df=df_clean[(df_clean[i]>up) | (df_clean[i]<lower)]
        df_clean=df_clean.drop(index=Outer_df.index)
    return df_clean

In [231]:
# 直接刪除任何含有缺失值的整行資料
# HINT：有更好的預處理填補方式嗎？

#刪掉完全重複
df_clean=df_clean.drop_duplicates()
#檢查空值
# print(df_clean.isnull().sum().sum())


#嘗試填入空值
"""df.info()"""
"""for i in range(3,8):
    df_clean.iloc[:,i]=df_clean.iloc[:,i].fillna(df.iloc[:,i].median())"""
#填入數字
"""df_clean['cylinders']=df_clean['cylinders'].fillna(df_clean['cylinders'].mode())
df_clean['origin']=df_clean['origin'].fillna(df_clean['origin'].mode())"""
"""df_clean.iloc[:,3:7]=df_clean.iloc[:,3:7].fillna(df_clean.iloc[:,3:7].median())"""
df_clean.iloc[:,4]=df_clean.iloc[:,4].fillna(df_clean.iloc[:,4].median())
# df_clean['name']=df_clean['name'].fillna(df_clean['name'].mode())
#去除極端值



# df_clean=STD(df_clean,'acceleration')

df_clean=STD(df_clean,'acceleration')

df_clean=STD(df_clean,'mpg',2.7)

df_clean=RemoveIqrOutliers(df_clean,'displacement')

df_clean=STD(df_clean,'horsepower',k=4)

df_clean=RemoveIqrOutliers(df_clean,'weight')
    
    #df_clean[i].iloc[Outer_df.index]=df_clean[i].mode()


#NULL檢查
"""print(df_clean.isnull())
missing_count = df_clean.isnull().sum().sum()
print(missing_count)"""

#標準化
# from sklearn.preprocessing import StandardScaler
# def ST(df):
#     scaler = StandardScaler()
#     df.loc[:,'cylinders':'model_year']=scaler.fit_transform(df.loc[:,'cylinders':'model_year'])
#     return df
# df_clean=ST(df_clean)
# print(df_clean['cylinders'].describe()['std'])



318
317
317
316
316
      id   mpg  cylinders  displacement  horsepower  weight  acceleration  \
0      3  16.0          8         304.0       150.0    3433          12.0   
1     18  27.0          4          97.0        88.0    2130          14.5   
2    376  37.0          4          91.0        68.0    2025          18.2   
3    248  36.1          4          91.0        60.0    1800          16.4   
4    177  23.0          4         115.0        95.0    2694          15.0   
..   ...   ...        ...           ...         ...     ...           ...   
313   71  19.0          3          70.0        97.0    2330          13.5   
314  106  12.0          8         350.0       180.0    4499          12.5   
315  270  21.1          4         134.0        95.0    2515          14.8   
316  348  37.7          4          89.0        62.0    2050          17.3   
317  102  26.0          4          97.0        46.0    1950          21.0   

     model_year  origin        name  ...  pontiac  rena

'print(df_clean.isnull())\nmissing_count = df_clean.isnull().sum().sum()\nprint(missing_count)'

In [232]:
data_dum = pd.get_dummies(df_clean['origin'])
data_dum=data_dum.drop('europe',axis=1)
df_clean=pd.concat([df_clean,data_dum],axis=1)
# print(df_clean)


# 3.特徵工程

In [233]:
# 定義要用來預測的特徵欄位
# HINT：這串列遺漏了部分特徵，另外，有方法額外加上新的特徵嗎？
features = ['weight', 'acceleration', 'model_year', 'cylinders', 'displacement', 'horsepower']
# features = ['weight', 'model_year', 'displacement','japan','usa','acceleration']
#找出name後面的colums的地址
i=(df_clean.columns.to_list().index('name'))
#將df_clean後面的加入features
comp=df_clean.iloc[:,i+1:].columns.to_list()
features+=comp
# 定義我們要預測的目標欄位
target = 'mpg'

# 從乾淨的資料中選取 X 和 y
X = df_clean[features]
y = df_clean[target]


# 4. 訓練模型

In [234]:
# 分割訓練集與測試集 (用於本地評估模型好壞)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 建立並訓練模型
model = LinearRegression()
model.fit(X_train, y_train)

# 進行預測與評估
train_predictions = model.predict(X_train)
test_predictions = model.predict(X_test)
train_rmse = np.sqrt(mean_squared_error(y_train, train_predictions))
test_rmse = np.sqrt(mean_squared_error(y_test, test_predictions))

print("模型已訓練完成！")
print(f"訓練誤差 (Train RMSE): {train_rmse:.4f} MPG")
print(f"測試誤差 (Test RMSE):  {test_rmse:.4f} MPG")
print("\n--- 模型學到的關係 ---")

# 各個特徵的權重
for feature, coef in zip(features, model.coef_):
    print(f"特徵 '{feature}' 的權重: {coef:.4f}")

模型已訓練完成！
訓練誤差 (Train RMSE): 2.8422 MPG
測試誤差 (Test RMSE):  3.6706 MPG

--- 模型學到的關係 ---
特徵 'weight' 的權重: -0.0061
特徵 'acceleration' 的權重: -0.0642
特徵 'model_year' 的權重: 3.3226
特徵 'cylinders' 的權重: -0.6571
特徵 'displacement' 的權重: 0.0211
特徵 'horsepower' 的權重: -0.0299
特徵 'amc' 的權重: -1.9977
特徵 'audi' 的權重: 0.4359
特徵 'bmw' 的權重: -1.5289
特徵 'buick' 的權重: 0.3465
特徵 'cadillac' 的權重: 0.0000
特徵 'capri' 的權重: -0.5452
特徵 'chevroelt' 的權重: -1.8155
特徵 'chevrolet' 的權重: 0.4470
特徵 'chevy' 的權重: -1.3482
特徵 'chrysler' 的權重: -2.9788
特徵 'datsun' 的權重: 2.4886
特徵 'dodge' 的權重: 0.1066
特徵 'fiat' 的權重: 1.3573
特徵 'ford' 的權重: -0.6873
特徵 'hi' 的權重: 2.4284
特徵 'honda' 的權重: 2.1804
特徵 'maxda' 的權重: -6.9853
特徵 'mazda' 的權重: -0.1430
特徵 'mercedes' 的權重: 0.7510
特徵 'mercedes-benz' 的權重: 1.2805
特徵 'mercury' 的權重: -1.4274
特徵 'oldsmobile' 的權重: 3.7203
特徵 'opel' 的權重: -0.5671
特徵 'peugeot' 的權重: 0.3088
特徵 'plymouth' 的權重: 0.9252
特徵 'pontiac' 的權重: 1.5411
特徵 'renault' 的權重: 1.9689
特徵 'saab' 的權重: -0.4055
特徵 'subaru' 的權重: 0.7988
特徵 'toyota' 的權重: 1.0640
特徵 'trium

In [235]:
print(df_clean)

      id   mpg  cylinders  displacement  horsepower  weight  acceleration  \
0      3  16.0          8         304.0       150.0    3433          12.0   
1     18  27.0          4          97.0        88.0    2130          14.5   
2    376  37.0          4          91.0        68.0    2025          18.2   
3    248  36.1          4          91.0        60.0    1800          16.4   
4    177  23.0          4         115.0        95.0    2694          15.0   
..   ...   ...        ...           ...         ...     ...           ...   
313   71  19.0          3          70.0        97.0    2330          13.5   
314  106  12.0          8         350.0       180.0    4499          12.5   
315  270  21.1          4         134.0        95.0    2515          14.8   
316  348  37.7          4          89.0        62.0    2050          17.3   
317  102  26.0          4          97.0        46.0    1950          21.0   

     model_year  origin        name  ...   saab  subaru  toyota  triumph  \

# 5.輸出提交檔案

In [236]:
# 讀取需要進行預測的測試檔案 test.csv
df_test = pd.read_csv("data/test.csv")

#NEW
data_dum = pd.get_dummies(df_test['origin'])
data_dum=data_dum.drop('europe',axis=1)
df_test=pd.concat([df_test,data_dum],axis=1)

#model_year分類
df_test=classify(df_test)


# 對測試資料進行預處理
# HINT：如果前面使用了其他的預處理方式，這邊要如何修改？
df_test=Firm(df_test)
# for i in (df_clean.columns):
#     if i not in df_test.columns:
#         df_test=pd.concat([df_test,pd.DataFrame({i:[False]*len(df_test)})],axis=1)


for i in (df_clean.columns):
    if i not in df_test.columns:
        df_test=pd.concat([df_test,pd.DataFrame({i:[False]*len(df_test)})],axis=1)

# print(df_test.tail(20))
#
df_test['cylinders']=df_test['cylinders'].fillna(df_test['cylinders'].mode())
df_test['origin']=df_test['origin'].fillna(df_test['origin'].mode())
df_test.iloc[:,3:7]=df_test.iloc[:,3:7].fillna(df_test.iloc[:,3:7].median())

#




#NEW2
# data_dum = pd.get_dummies(df_test['name'])
# df_test=pd.concat([df_test,data_dum],axis=1)
# 使用訓練好的模型，對測試資料進行預測
predictions = model.predict(df_test[features])

# 建立一個新的 DataFrame
submission_df = pd.DataFrame({'Id':df_test['id'], 'mpg': predictions})

# 保存為 submission.csv
submission_df.to_csv('submission.csv', index=False)
print("提交文件 'submission.csv' 已成功生成！")

提交文件 'submission.csv' 已成功生成！


# 6. 報告

姓名：_____陸竑甫_____ 學號：_____114409529_____

第一部分：準確度分數 (Accuracy Scores) (1分)  
我的準確度分數：____2.68218______  

第二部分：我的實驗記錄 (My Experiment Log) (3分)  
請記錄你做了哪些嘗試來提升分數，至少記錄兩次不同的嘗試。 
【實驗 1】  
    我做的修改：__標準化_ 
    結果與觀察 (分數變化、心得等)：_____完全沒變_____  
    該次實驗分數： _____3.55541_______ 
【實驗 2】  
    我做的修改：__將國家one hot encoding 分為不同群_ 
    結果與觀察 (分數變化、心得等)：_____分數十分顯著的提升了許多_____  
    該次實驗分數： _____我幾乎忘記了，那是有點久的事了，我記得大概是3.34631(從kaggle找的)_______  
【實驗 3】  
    我做的修改：__將horsepower drop掉_ 
    結果與觀察 (分數變化、心得等)：_____分數有下降了許多，但之後還是加了回來(見實驗五)_____  
    該次實驗分數： _____這個我真的忘了_______  
【實驗 4】  
    我做的修改：__將name裡的製造商切割出來並使用one hot encoding 分為不同群_ 
    結果與觀察 (分數變化、心得等)：_____分數十分顯著的下降了許多，而且相比於直接將所有name one hot 來的有用許多_____  
    該次實驗分數： _____我是一值改依值改疊上去的，所以沒有單獨測只有這個的影響，不過此時分數是2.92614_______  
【實驗 5】  
    我做的修改：___刪除異常值，嘗試了兩種方式，四分位數法和標準差法，_______________________________________________________________________________  
    結果與觀察 (分數變化、心得等)：____最後結果就是有些使用四分位數有些使用標準差法，且每個取值範圍都不一樣。一個一個測看看分數有沒有降。講真的心滿累的。最後終於過了Medium Baseline______________________________________________________________  
    該次實驗分數： ____2.68218________  

第三部分：總結與心得 (Conclusion & Reflection) (2分)  
請撰寫一段約 50-100 字的心得總結。內容需包含：  
(1) 你認為本次實驗中，提升準確率最有效的修改是什麼。  
(2) 這次不斷嘗試與修正的過程，帶給你最大的學習與啟發。  
內容：_______講真的，train到後來已經基本上不知道自己在幹嘛了，一直改一些奇怪的數字直到分數變低，有些one hot之類的內容還能理解為甚麼這樣做能讓分數降低，但有些操作真的就只是抱持著試試看，反正也沒別的能改了。像是df_clean=STD(df_clean,'horsepower',k=4)這之中的k=4
就是完全隨便設的，沒想到終於過了Medium Baseline。我搞這個搞了超過12個小時，看到過了的那一刻真的快哭了QQ從競賽網站應該也能看到我總共上傳了33次，講真的我做過的測試遠遠不止上面寫的那些，只是有些太沒用我刪完就忘了。有的我只有註解掉，所以很有可能在程式碼的註解裡面翻到也說不定_______________________________________    